In [ ]:
#|default_exp cnn_virus.architecture

In [ ]:
#|hide
from __future__ import annotations

import nbdev
import os

from ecutilities.ipython import nb_setup
from fastcore.test import *
from metagentools.cnn_virus.utils import setup_nb
from nbdev.showdoc import *

In [ ]:
#| hide
nb_setup()
ON_COLAB, p2dataroot, p2data = setup_nb(_dev=True)
if not ON_COLAB: os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

Set autoreload mode
Running locally


In [ ]:
#|export
# Imports all dependencies
import tensorflow as tf
import tensorflow.keras

from pathlib import Path
from typing import Callable, Tuple
from metagentools.core import ProjectFileSystem

from tensorflow.keras.layers import Convolution1D, Dense, Flatten, Dropout, Activation, BatchNormalization, Input
from tensorflow.keras.layers import MaxPooling1D, Concatenate
from tensorflow.keras.models import Sequential, Model, load_model

In [ ]:
#|hide
print(f"Tensorflow version: {tf.__version__} - Expected 2.8.2")

Tensorflow version: 2.8.2 - Expected 2.8.2


# architecture

> Architectures used for this project

The paper "A multi-task CNN learning model for taxonomic assignment of human viruses" proposes a CNN model applied to virus. 

The module `architecture` includes:
- Our reproduction of the original paper model using the latest version of Keras
- All other variants we are developing


> "A multi-task CNN learning model for taxonomic assignment of human viruses"
>
> by Haoran Ma, Tin Wee Tan and Kenneth Hon Kim Ban
>
> From 19th International Conference on Bioinformatics 2020 (InCoB2020) - Virtual. 25-29 November 2020


## Original Architecture

In [ ]:
#| export

def create_model_original(
    load_parameters: bool = True, # Load pretrained weights when True
    path2parameters: Path = None,  # Path to pretrained weights, defaults to project CNN Virus weights
    ) -> tf.keras.Model:          # New instance of an original paper architecture
    """Build a CNN model as per CNN Virus paper"""

    print("Creating CNN Model (Original)")
    # Build cnn model
    input_seq=Input(shape=(50,5), name='input-seq')
    layer1=Convolution1D(512, 5, padding="same",activation="relu",kernel_initializer="he_uniform", name="conv-1")(input_seq)
    layer2=BatchNormalization(momentum=0.6, name='bn-1')(layer1)
    layer3=MaxPooling1D(pool_size=2,padding='same', name='maxpool-1')(layer2)
    layer4=Convolution1D(512, 5, padding="same",activation="relu",kernel_initializer="he_uniform", name="conv-2")(layer3)
    layer5=BatchNormalization(momentum=0.6, name='bn-2')(layer4)
    layer6=MaxPooling1D(pool_size=2,padding='same',name='maxpool-2')(layer5)
    layer7=Convolution1D(1024, 7, padding="same", activation="relu",kernel_initializer="he_uniform", name="conv-3")(layer6)
    layer8=Convolution1D(1024, 7, padding="same", activation="relu",kernel_initializer="he_uniform", name="conv-4")(layer7)
    layer9=BatchNormalization(momentum=0.6, name='bn-3')(layer8)
    layer10=MaxPooling1D(pool_size=2,padding='same', name='maxpool-3')(layer9)
    layer11=Flatten(name='flatten')(layer10)
    layer12=Dense(1024,kernel_initializer="he_uniform", name='dense-1')(layer11)
    layer13=BatchNormalization(momentum=0.6, name='bn-4')(layer12)
    layer14=Dropout(0.2, name='do-1')(layer13)
    labels=Dense(187, activation='softmax',kernel_initializer="he_uniform",name="labels")(layer14)
    output_con=Concatenate(name='concat')([layer14,labels])
    layer15=Dense(1024, kernel_initializer="he_uniform", name='dense-2')(output_con)
    layer16=BatchNormalization(momentum=0.6, name='bn-5')(layer15)
    pos=Dense(10, activation='softmax',kernel_initializer="he_uniform",name="pos")(layer16)
    model = Model(inputs=input_seq, outputs=[labels,pos], name="CNN_Virus")

    # Load pretrained weights
    if load_parameters:
        if path2parameters is None: 
            path2parameters = ProjectFileSystem().data /'saved/cnn_virus_original/pretrained_model.h5'
        if not path2parameters.is_file(): 
            raise FileNotFoundError(f"Could not find pretrained model at {path2parameters}")
        print(f"Loading parameters from {path2parameters.name}")
        model.load_weights(path2parameters)
        print("Created pretrained model")
    else:
        print("Created randomly initialized model")
    return model    

This is the original paper's model, taking 50-mer sequences and predicting:

1. the logits for 187 viruses
2. the logits for one of 10 regions in the original virus genome

In [ ]:
model = create_model_original(load_parameters=False)

Creating CNN Model (Original)
Created randomly initialized model


In [ ]:
model = create_model_original(load_parameters=True)

Creating CNN Model (Original)
Loading parameters from pretrained_model.h5
Created pretrained model


In [ ]:
model.summary()

Model: "CNN_Virus"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input-seq (InputLayer)         [(None, 50, 5)]      0           []                               
                                                                                                  
 conv-1 (Conv1D)                (None, 50, 512)      13312       ['input-seq[0][0]']              
                                                                                                  
 bn-1 (BatchNormalization)      (None, 50, 512)      2048        ['conv-1[0][0]']                 
                                                                                                  
 maxpool-1 (MaxPooling1D)       (None, 25, 512)      0           ['bn-1[0][0]']                   
                                                                                          

In [ ]:
#| hide
nbdev.nbdev_export()